# Transformée en Ondelettes 2D, application au traitement des images

### <Font color = "blue"> ENGELSEN Gorm et RAHAL Skander

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests
import boto3
import os
import math

In [ ]:
s3_endpoint_url = 'https://object-rook-ceph.apps.math.cnrs.fr/'
s3_access_key_id = '9F7EB8YBUWXDV7A4IZYW' # le contenu de secrets/dossal
s3_secret_access_key = 'skV01Eei5M3xVOxROIDr3qymYhWtkrxPpMyj8nwb' # le contenu de secrets/dossal
s3_bucket = 'signal-image'
s3 = boto3.client('s3',
                  '',
                  endpoint_url = s3_endpoint_url,
                  aws_access_key_id = s3_access_key_id,
                  aws_secret_access_key = s3_secret_access_key)
Data=["Lenna.jpg","Canaletto.jpeg","MinotaureBruite.jpeg","Cartoon.jpg"]
if not os.path.isfile('Lenna.jpg'):
    for filenames in Data:  
        s3.download_file(s3_bucket,filenames,filenames)
def chargeData(name):
    if name=='Lenna':
        res=np.array(Image.open("Lenna.jpg")).astype(float)
    if name=='Canaletto':
        res=np.array(Image.open("Canaletto.jpeg")).astype(float)
    if name=='Minotaure':
        res=np.array(Image.open("MinotaureBruite.jpeg")).astype(float)  
    if name=='Cartoon':
        res=np.array(Image.open("Cartoon.jpg")).astype(float) 
    return res
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
options2=dict(width=700,height=400,xaxis=None,yaxis=None,toolbar=None)

## Approximation linéaire et non linéaire.

In [ ]:
im2=chargeData('Lenna')
im=chargeData('Canaletto')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
size=400
WT= pywt.wavedecn(im, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray',width=size,height=size)

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def ApproxOnd2D(S,qmf,L,threshold):
    Lmax=pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len) 
    L1=min(L,Lmax)
    WT= pywt.wavedecn(S, qmf, mode='per', level=L1)
    arr, coeff_slices = pywt.coeffs_to_array(WT) 
    WTS=arr*(np.abs(arr)>threshold)
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices) 
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    ncoeffs = len(np.where(WTS !=0)[0])
    return Srec,ncoeffs


In [ ]:
#test
Srec,ncoef=ApproxOnd2D(im2,'Haar',9,20)
print(ncoef)
pn.Row(hv.Image(im,label="image originale").opts(**options),hv.Image(Srec,label="image après approximation linéaire").opts(**options))

<font color='green'>On remarque que l'image obtenue après approximation linéaire est de moins bonne qualité que l'image originale car on ne conserve que des coefficients d'ondelette supérieurs à un certain threshold.</font>

Ecrire une focntion PSNR.

In [ ]:
def PSNR(I,Iref):
    temp=I.ravel()
    tempref=Iref.ravel()
    NbP=I.size
    EQM=np.sum((temp-tempref)**2)/NbP
    b=np.max(np.abs(tempref))**2
    return 10*np.log10(b/EQM)

In [ ]:
psnr=PSNR(im2,Srec)
print(psnr)

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2nonlin(I,qmf,L,N):
    n,m=np.shape(I)
    Lmax=pywt.dwt_max_level(len(I),pywt.Wavelet(qmf).dec_len)
    L1=min(L,Lmax)
    WT= pywt.wavedecn(I, qmf, mode='per', level=L1)
    arr, coeff_slices = pywt.coeffs_to_array(WT)
    arr_vector=arr
    arr_vector=arr_vector.flatten()
    N1=len(arr_vector)
    Ind=np.argsort(np.abs(arr_vector))
    WTS=np.zeros(N1)
    WTS[Ind[N1-N:N1]]=arr_vector[Ind[N1-N:N1]]
    WTS=WTS.reshape(n,m)
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Irec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    p=PSNR(I,Irec)
    return Irec,p

In [ ]:
Irec,p=ApproxOnd2nonlin(im,'db2',6,5000)
pn.Row(hv.Image(im,label="image originale").opts(**options),hv.Image(Irec,label="image après approximation non linéaire").opts(**options))


<font color='green'>On remarque que l'image obtenue après approximation non linéaire est de moins bonnes qualité que l'image originale comme dans la cas de l'approximation linéaire car on ne conserve qu'un nombre fini des plus grands coefficients d'ondelettes (ici N=5000). Ici on utilise la base de daubechies ,c'est pour cela qu'on obtient pas le même résultat que pour la base de haar.</font>

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
wavelist=['haar','db2','db3','db4','coif1','coif2','coif3']

In [ ]:
class Approxnonlin2D(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,7))
    N = param.Integer(2000,bounds=(1,10000))
  #  @param.depends('wave', 'N', 'L')
    def view(self):
        Iref=imagesRef[self.image]
        Irec,p = ApproxOnd2nonlin(Iref,self.wave,self.L,self.N)
        Image_originale=hv.Image(Iref,label="image originale").opts(cmap='gray',width=300,height=300)
        Image_aprox=hv.Image(Irec,label="image approximation non linéaire").opts(cmap='gray',width=300,height=300)
        return pn.Row(pn.Column(Image_originale,'PSNR pour approximation non linéaire',p),pn.Column(Image_aprox))
       

In [ ]:
approxno2D = Approxnonlin2D()
pn.Row(approxno2D.param,approxno2D.view)

<font color='green'> On remarque dans le cas d'une approximation non linéaire, lorsqu'on augmente la valeur de N,la valeur de PSNR est beaucoup plus importante et la qualité d'image s'améliore lorsque la valeur de celle-ci augmente.
De plus, dans le cadre des ondelettes de daubechies , la velur de PSNR est plus élevée comparant aux ondelettes de Haar.</font>

In [ ]:
class Approxlin2D(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,7))
    T = param.Number(1,bounds=(1,100))
  #  @param.depends('wave', 'N', 'L')
    def view(self):
        Iref=imagesRef[self.image]
        Irec,ncoef = ApproxOnd2D(Iref,self.wave,self.L,self.T)
        p=PSNR(Iref,Irec)
        Image_originale=hv.Image(Iref,label="image originale").opts(cmap='gray',width=300,height=300)
        Image_aprox=hv.Image(Irec,label="image approximation linéaire").opts(cmap='gray',width=300,height=300)
        return pn.Row(pn.Column(Image_originale,'PSNR pour approximation linéaire',p),pn.Column(Image_aprox,'nombre de coefficients',ncoef))
       

In [ ]:
approx2D = Approxlin2D()
pn.Row(approx2D.param,approx2D.view)

<font color='green'>Le coefficient L (profondeur de la transformée en ondelettes) a peu d'influence sur l'image obtenue après approximation non linéaire.
Cependant, le coefficient de seuillage T agit sur la résultante. En effet , on remarque que la valeur de PSNR varie fortement en modifiant la valeur de T.
On remarque aussi qu'avec une base de Haar, le PSNR ainsi que le nombre de coefficients sont beaucoup plus faibles que dans le cadre des ondelettes de Daubechies.( C'est pourcela la qualité d'image est meilleure dans le cadre des ondelettes de Daubechies par rapport aux ondelettes de Haar pour des valeurs de T et L fixées)</font>

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2nonlin

In [ ]:
import itertools
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image':imagesRef,'N':np.linspace(1000,50000,30),'wave':wavelist}
dfexp=pd.DataFrame(data=itertools.product(*experiences.values()),columns=experiences.keys())


In [ ]:
print(dfexp)

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:
def row2PSNR(row):
    L=7
    qmf=row[2]
    N=int(row[1])
    if row[1]=='Lenna':
        Irec,p=ApproxOnd2nonlin(im2,qmf,L,N)
    else:
        Irec,p=ApproxOnd2nonlin(im,qmf,L,N)
    return {'PSNR':p}
    

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
row = dfexp.iloc[0,:] 
p=row2PSNR(row)
print(p)

In [ ]:
res=dfexp.apply(row2PSNR,axis='columns')
print(res)

In [ ]:
dfexp=dfexp.assign(PSNR=pd.DataFrame.from_records(res))

In [ ]:
print(dfexp)

Utiliser hvplot pour visualiser la base de données.

In [ ]:
import hvplot.pandas

In [ ]:
from bokeh.models import HoverTool
h = HoverTool()
dfexp.hvplot('N','PSNR',by='wave',kind='scatter',groupby=['Image'],label = "Tracé du PSNR en fonction du nombre de coefficients \n d'ondelettes pour différentes bases d'ondelettes \n \n" )\
.opts(width=600,tools = [h]).redim.range(PSNR=(20,45),N=(1000.0,50000.0))

<font color='green'>On remarque que lorsqu'on augmente la valeur de N , la valeur de PSNR augmente aussi.
De plus , on remarque que les ondelettes de coifmann donnent la meilleure approximation contrairement aux ondelettes de haar qui sont les moins bonnes car admettant un PSNR plus faible que les autres.</font>



## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
def SeuillageDurOndelettes(I,qmf,L,Seuil):
    Lmax=pywt.dwt_max_level(len(I),pywt.Wavelet(qmf).dec_len) #level de la TFO
    L=min(L,Lmax)
    WTB= pywt.wavedecn(I, qmf, mode='per', level=L)
    arr, coeff_slices,coeff_shapes = pywt.ravel_coeffs(WTB)
    WTS=arr*(np.abs(arr)>Seuil)
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices,coeff_shapes)
    Irec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')

    return Irec

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
class WaveSeuillage(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(10,bounds=(1,1000))
    def view(self):
        Iref = imagesRef[self.image]
        Irec = SeuillageDurOndelettes(Iref,self.wave,self.L,self.Seuil)
        
        Ioriginale = hv.Image(Iref,label="Image originale").opts(cmap='gray',width=350,height=350)
        Iapprox = hv.Image(Irec,label="Approximation non linéaire \n par seuillage dur").opts(cmap='gray',width=350,height=350)
        

        P = PSNR(Iref,Irec)
        return pn.Row(Ioriginale,pn.Column (Iapprox, "PSNR de l'image par approximation non linéaire : ",P))

In [ ]:
dash3=WaveSeuillage()
pn.Row(dash3.param,dash3.view)

<font color='green'>Comme dans les autres cas, la valeur de L agit de manière très faible sur l'image.
Lorsqu'on augmente la valeur du seuil ,l'image se dégrade de plus en plus car la valeur de PSNR diminue.
Les ondelettes de Haar donnent des valeurs PSNR plus faibles que les ondelettes de daubechies et celles de coifman vu que celles ci sont des fonctions constantes par morceau (on remarque des rectangles prononcés sur l'image lorsque la valeur du seuil est élevée).</font>

In [ ]:
def Bruit(im,sigma,seednoise):
    np.random.seed(seed=seednoise)
    n1,n2=np.shape(im)
    B=np.random.randn(n1,n2)
    ib=im+sigma*B
    ib=np.clip(ib,0,255)
    return ib

In [ ]:
ib = Bruit(im,100,2)
pn.Row(hv.Image(im,label="Image originale").opts(cmap='gray',width=300,height=300),hv.Image(ib,label="Image bruitée").opts(cmap='gray',width=300,height=300))

In [ ]:
def Debruit(im,qmf,L,T,seednoise,sigma): 

    n1,n2=np.shape(im)
  
    Lmax=pywt.dwt_max_level(len(im),pywt.Wavelet(qmf).dec_len) #level de la TFO
    L=min(L,Lmax)
     
    ib = Bruit(im,sigma,seednoise)

    Seuil=T*sigma
    Irec=SeuillageDurOndelettes(ib,qmf,L,Seuil)
    
    psnr1=PSNR(ib,im) 
    psnr2=PSNR(Irec,im)
    return Irec,ib,psnr1,psnr2 

Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitéeset débruitées.

In [ ]:
class WaveDebruit(param.Parameterized):
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys()) 
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,100)) 
    seednoise = param.Integer(1,bounds=(0,50))
    T=param.Number(3,bounds=(0,8))
    def view(self):
        S = imagesRef[self.image]
        Srec,SB,p1,p2=Debruit(S,self.wave,self.L,self.T,self.seednoise,self.Sigma)
             
    
        Ioriginal = hv.Image(S, label = "Image orinigale").opts(cmap='gray',width=350,height=350)
        Ibuite = hv.Image(SB,label = "Image bruitée").opts(cmap='gray',width=350,height=350) 
        Idebruite = hv.Image(Srec,label = "Image débruitée").opts(cmap='gray',width=350,height=350)
        
        return pn.Column(pn.Row(Ioriginal,pn.Column (Ibuite, "PSNR entre image bruitée et image originale: ",p1),pn.Column (Idebruite, "PSNR entre image débruitée et image originale: ",p2)))        

In [ ]:
wavedebruit = WaveDebruit()
pn.Row(wavedebruit.param,wavedebruit.view)

<font color='green'>On remarque que le coefficient L a peu d'influence sur la qualité de l'approximation comme dans les autres cas.
De plus , lorsqu'on ajoute du bruit à une image , la qualité de celle-ci diminue ce qui explique que la valeur de PSNR est plus faible pour une image bruité comparant à une image non bruitée.</font>

## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTrans(im,qmf,L,T,seednoise,sigma,NbT):
    
    n1,n2=np.shape(im)
    
     #profondeur maximal de la TFO
    Lmax=pywt.dwt_max_level(len(im),pywt.Wavelet(qmf).dec_len) #level de la TFO
    L=min(L,Lmax)
    
    # bruitage de l'image : 
    ib = Bruit(im,sigma,seednoise)
    
    # débruitage de l'image
    Seuil=T*sigma
    ISum=0*ib
    P=np.zeros(NbT)
    
    for k in np.arange(0,NbT): 
        #Translation des lignes et des colonnes de l'image
        SBtemp=np.roll(ib,k,axis = 1) #décalage des colonnes
        SBtemp=np.roll(SBtemp,k,axis = 0) #décalage des lignes
        #Seuillage sur l'image translatée
        Srectemp=SeuillageDurOndelettes(SBtemp,qmf,L,Seuil)
        #Translation des lignes et des colonnes inverse
        Srectemp2=np.roll(Srectemp,-k,axis = 1)
        Srectemp2=np.roll(Srectemp2,-k,axis = 0)      

        ISum=ISum+Srectemp2
        Srec=ISum/(k+1)
        P[k]=PSNR(Srec,im) #PSNR(I,Iref) de chaque translation
    P_moy = np.mean(P) #moyenne des PNSR des images reconstruite 
    P_Buite=PSNR(ib,im) #PSNR(I,Iref) PNSR image bruitée
    
    return Srec,ib,P,P_moy,P_Buite

Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

In [ ]:
class Debruit_translat(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    NbT = param.Integer(4,bounds=(1,8))
    Sigma = param.Number(10,bounds=(1,100))
    seednoise = param.Integer(1,bounds=(0,50))
    L = param.Integer(7,bounds=(0,7))
    T=param.Number(3,bounds=(0,8))
    def view(self):
        S=imagesRef[self.image]
        Srec,SB,P,P_moy,P_Buite=DebruitTrans(S,self.wave,self.L,self.T,self.seednoise,self.Sigma,self.NbT)
        
        #tracé courbe PNSR en fonction de la translation
        PSNR = hv.Curve(P,"Translation","PSNR", label = "Tracé des valeurs de PSNR \n pour différentes translations").opts(width=300,height=300)
        
        #définition des images        
        Ioriginal = hv.Image(S, label = "Image orinigale").opts(cmap='gray',width=350,height=350)
        Ibuite = hv.Image(SB,label = "Image bruitée").opts(cmap='gray',width=350,height=350) 
        Idebruite = hv.Image(Srec,label = "Image débruitée").opts(cmap='gray',width=350,height=350)
        
        return pn.Column(PSNR, pn.Row(Ioriginal,pn.Column (Ibuite, "PSNR entre image bruitée et image originale: ",P_Buite)
                                ,pn.Column (Idebruite, "PSNR moyen (sur les translations) entre \n image débruitée et image originale: ",P_moy)))        

In [ ]:
DebruitTranslat = Debruit_translat()
pn.Row(DebruitTranslat.param,DebruitTranslat.view)

## Débruitage d'une image couleur.

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissnaces de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vraiu problème, il faut juste que les dimensions soit des multiples de puissances de 
2. C'est la puissance de 2 qui définira l'échalle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

In [ ]:
Mi = chargeData('Minotaure')
Minotaure=np.clip(Mi,0,255)
hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=400)

In [ ]:
def DebruitTrans_Couleurs(im,qmf,L,T,sigma,NbT):
    n1,n2,n3=np.shape(im)
    Seuil=T*sigma
    ISum=np.zeros(np.shape(im))
    Srec=np.zeros(np.shape(im))
    P=np.zeros(NbT)
    for j in range(n3):
        ib=im[:,:,j]
        for k in np.arange(0,NbT):
            #Translation des lignes et des colonnes de l'image
            SBtemp=np.roll(ib,k,axis = 1) #décalage des colonnes
            SBtemp=np.roll(SBtemp,k,axis = 0) #décalage des lignes
            #Seuillage sur l'image translatée
            Srectemp=SeuillageDurOndelettes(SBtemp,qmf,L,Seuil)
            #Translation des lignes et des colonnes inverse
            Srectemp2=np.roll(Srectemp,-k,axis = 1)
            Srectemp2=np.roll(Srectemp2,-k,axis = 0)      
            ISum[:,:,j]=ISum[:,:,j]+Srectemp2
            Srec[:,:,j]=ISum[:,:,j]/(k+1)
    return Srec

In [ ]:
def Resize_debruit(ib,qmf,L,T,sigma,NbT): 
    n1,n2,n3 = np.shape(ib) 
      
    if n1 < n2 : 
        CaSym = np.zeros((max(n1,n2),max(n1,n2),n3))
        ib_ =  np.flip(ib,axis=0)[:(n2-n1),:,:] 
        CaSym[n1::,:,:] = ib_
        CaSym[:n1,:,:] = ib
        
        Srec = DebruitTrans_Couleurs(CaSym,qmf,L,T,sigma,NbT)[:n1,:,:]
    
    if n1 > n2 : 
        CaSym = np.zeros((max(n1,n2),max(n1,n2),n3))
        result = np.zeros((max(n1,n2),max(n1,n2),n3))
        ib_ =  np.flip(ib,axis=1)[:,:(n1-n2),:] 
        CaSym[:,n2::,:] = ib_
        CaSym[:,:n2,:] = ib     
        
        Srec = DebruitTrans_Couleurs(CaSym,qmf,L,T,sigma,NbT)[:,:n2,:] 
    
    if n1 == n2 : 
        Srec = DebruitTrans_Couleurs(ib,qmf,L,T,sigma,NbT)
    
    return Srec

In [ ]:
class Debruit_translat_Minotaure(param.Parameterized):
    wave = param.ObjectSelector(default="db2",objects=wavelist)
    NbT = param.Integer(4,bounds=(1,8))
    Sigma = param.Number(10,bounds=(1,100))
    L = param.Integer(7,bounds=(0,7))
    T=param.Number(3,bounds=(0,8))
    def view(self):
        S = Mi
        Minotaure=np.clip(Mi,0,255)
        Srec=Resize_debruit(S,self.wave,self.L,self.T,self.Sigma,self.NbT)
        MinotaureRec=np.clip(Srec,0,255)
        
        #définition des images :    
        Ibruite = hv.RGB(Minotaure.astype('uint8'),label="Image bruitée originale").opts(xlabel=None,ylabel=None,width=350,height=350)
        Idebruite = hv.RGB(MinotaureRec.astype('uint8'),label = "Image débruitée").opts(xlabel=None,ylabel=None,width=350,height=350)
    
        return pn.Column(pn.Row(Ibruite,Idebruite))

In [ ]:
DebruitTranslatMinotaure = Debruit_translat_Minotaure()
pn.Row(DebruitTranslatMinotaure.param,DebruitTranslatMinotaure.view)

In [ ]:
deb=DebruitTrans_Couleurs(Minotaure,'db2',7,4.2,30,8)
hv.RGB(deb.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=400)

<font color='green'>Ceci est un exemple d'une image minotaure débruitée initialement bruitée.</font>

Rédiger également une fonction prenant en entrée un nom de fichier 
permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question.
On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

### Plan d'expériences pour évaluer l'impact des translations  

Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

In [ ]:
experiences_DebruitTrans = {'Image':imagesRef.keys(),'NbT':np.arange(1,5),'wave':wavelist,'Sigma':np.linspace(10,30,2)}
...

In [ ]:
print(dfexp_DebruitTrans)

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
def Debruit_Translat_PSNRMoyen(I,wave,sigma,NbT,n):
    ...

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row):
    

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value,counts = np.unique(x, return_counts=True)
    Proba=counts/len(x)
    Ent=-np.sum(np.log2(Proba)*Proba)
    return Ent

In [ ]:
x=np.array([13,13,2,7,13,7,1,13])
print(ShannonEntropy(x))

In [ ]:
y=np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

In [ ]:
def QuantificationOndelettes(I,qmf,Pas):
    

Créer le dashboard asscoié à la focntion précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
class WaveQuant(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    QS = param.Number(30,bounds=(10,300))
    ...

Créer dun plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

In [ ]:
experiences_quant = {'QS':np.linspace(30,200,10),'wave':wavelist}
...

In [ ]:
def row2DistorsionRate(row):
    

## Pour aller plus loin (à titre informatif et optionnel)

Nous proposons ici d'effectuer la compression sur les 3 canau RGB. Or l'oeil humain est plus sensible à la luminance qu'aux composantes purement chromatiques. C'est pourquoi, la plupart des algorithmes de compressions sont effectué dans un espace colorimétrique YUV où Y est la luminance. On alloue alors plus d'information au canal Y et on comprime plus drastiquement les deux autres canaux. Une méthode standart consiste par exemple à sous-échantionner d'un facteur 2 les deux composantes U et V avant de les comprimer. 

https://fr.wikipedia.org/wiki/Sous-échantillonnage_de_la_chrominance

On obtient alors des images de chrominances moins résolues et donc moins lourdes mais le rendu final reste correct car l'oeil humain est nettement plus sensible à la luminance. 